In [1]:
import pandas as pd
import numpy as np

print("✅ Libraries imported")


✅ Libraries imported


In [2]:
# Load dataset 
df = pd.read_csv("email_phishing_dataset_FINAL.csv")

print("✅ Dataset loaded successfully")

✅ Dataset loaded successfully


In [3]:
# View first few rows
df.head()

,email_subject_len,email_has_urgent_keyword,email_from_domain,web_url,web_url_len,web_ip_add,web_geo_loc,web_tld,web_who_is,web_https,...,content_entropy,domain_trust_score,email_domain_matches_url,email_url_domain_similarity,content_num_forms,content_num_inputs,content_num_scripts,content_suspicious_keywords,semantic_coherence_score,brand_consistency_score
0,32,0,spamassassin.zones.apache.org,http://tools.ietf.org/html/rfc1583,34,30.180.42.35,United States,org,complete,yes,...,4.620961,0.8,0,0.411765,0,0,5,0,0.043153,0.5
1,46,0,gmail.com>,http://www.quickfixgolf.com,27,150.66.16.42,Japan,com,complete,yes,...,4.742243,0.8,0,0.500000,0,0,2,0,0.081125,0.5
2,21,0,telefonica.net>,http://www.lvnazarene.org,25,180.123.185.229,China,org,complete,yes,...,4.663432,0.8,0,0.400000,0,0,2,0,0.000000,0.5
3,99,1,gmail.com>,http://hatchersmartialarts.homestead.com/front...,51,46.97.122.170,Romania,com,complete,yes,...,4.971977,0.8,0,0.466667,0,1,2,2,0.071720,0.5
4,72,1,luebeck.de>,http://www.gabile.com/,22,94.145.85.24,Denmark,com,incomplete,no,...,4.338266,0.4,0,0.357143,0,1,24,3,0.000000,0.5


In [4]:
# Dataset shape
print("Dataset shape:", df.shape)


Dataset shape: (8000, 39)


In [5]:
# Check data types
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   email_subject_len            8000 non-null   int64  
 1   email_has_urgent_keyword     8000 non-null   int64  
 2   email_from_domain            7822 non-null   object 
 3   web_url                      8000 non-null   object 
 4   web_url_len                  8000 non-null   int64  
 5   web_ip_add                   8000 non-null   object 
 6   web_geo_loc                  8000 non-null   object 
 7   web_tld                      8000 non-null   object 
 8   web_who_is                   8000 non-null   object 
 9   web_https                    8000 non-null   object 
 10  web_js_len                   8000 non-null   float64
 11  web_js_obf_len               8000 non-null   float64
 12  web_content                  8000 non-null   object 
 13  domain_age        

In [3]:
# email_from_domain missing values
df = df.dropna(subset=['email_from_domain'])

print("Shape after dropping email_from_domain missing:", df.shape)


Shape after dropping email_from_domain missing: (7822, 39)


In [4]:
# Create missingness indicator
df['domain_age_missing'] = df['domain_age'].isnull().astype(int)

# Impute missing domain_age with 0 (unknown/new domain)
df['domain_age'] = df['domain_age'].fillna(0)

print("Missing domain_age after handling:", df['domain_age'].isnull().sum())


Missing domain_age after handling: 0


In [8]:
print("\nFinal dataset shape:", df.shape)
print("\nAny remaining missing values?")
print(df.isnull().sum().sum())



Final dataset shape: (7822, 40)

Any remaining missing values?
0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7822 entries, 0 to 7999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   email_subject_len            7822 non-null   int64  
 1   email_has_urgent_keyword     7822 non-null   int64  
 2   email_from_domain            7822 non-null   object 
 3   web_url                      7822 non-null   object 
 4   web_url_len                  7822 non-null   int64  
 5   web_ip_add                   7822 non-null   object 
 6   web_geo_loc                  7822 non-null   object 
 7   web_tld                      7822 non-null   object 
 8   web_who_is                   7822 non-null   object 
 9   web_https                    7822 non-null   object 
 10  web_js_len                   7822 non-null   float64
 11  web_js_obf_len               7822 non-null   float64
 12  web_content                  7822 non-null   object 
 13  domain_age             

In [5]:
# ENCODING NON-NUMERICAL FEATURES

df_encoded = df.copy()

# 1️⃣ email_from_domain → frequency encoding
domain_freq = df_encoded['email_from_domain'].value_counts(normalize=True)
df_encoded['email_domain_freq'] = df_encoded['email_from_domain'].map(domain_freq)
df_encoded['email_domain_freq'] = df_encoded['email_domain_freq'].fillna(0)
df_encoded.drop(['email_from_domain'], axis=1, inplace=True)

# 2️⃣ web_geo_loc → frequency encoding
geo_freq = df_encoded['web_geo_loc'].value_counts(normalize=True)
df_encoded['geo_freq'] = df_encoded['web_geo_loc'].map(geo_freq).fillna(0)
df_encoded.drop(['web_geo_loc'], axis=1, inplace=True)

# 3️⃣ web_tld → trust score
trusted_tlds = ['com', 'org', 'net', 'edu', 'gov']
df_encoded['tld_trust_score'] = df_encoded['web_tld'].apply(
    lambda x: 1.0 if x in trusted_tlds else 0.0
)
df_encoded.drop(['web_tld'], axis=1, inplace=True)

# 4️⃣ web_who_is → binary
df_encoded['web_who_is'] = df_encoded['web_who_is'].map({'complete': 1, 'incomplete': 0})

# 5️⃣ web_https → binary
df_encoded['web_https'] = df_encoded['web_https'].map({'yes': 1, 'no': 0})

# 6️⃣ Drop noisy features
df_encoded.drop(['web_url', 'web_ip_add', 'web_content'], axis=1, inplace=True)

print("✅ Encoding complete")
print("Final shape:", df_encoded.shape)


✅ Encoding complete
Final shape: (7822, 37)


In [11]:
df.head()

,email_subject_len,email_has_urgent_keyword,email_from_domain,web_url,web_url_len,web_ip_add,web_geo_loc,web_tld,web_who_is,web_https,...,domain_trust_score,email_domain_matches_url,email_url_domain_similarity,content_num_forms,content_num_inputs,content_num_scripts,content_suspicious_keywords,semantic_coherence_score,brand_consistency_score,domain_age_missing
0,32,0,spamassassin.zones.apache.org,http://tools.ietf.org/html/rfc1583,34,30.180.42.35,United States,org,complete,yes,...,0.8,0,0.411765,0,0,5,0,0.043153,0.5,0
1,46,0,gmail.com>,http://www.quickfixgolf.com,27,150.66.16.42,Japan,com,complete,yes,...,0.8,0,0.500000,0,0,2,0,0.081125,0.5,0
2,21,0,telefonica.net>,http://www.lvnazarene.org,25,180.123.185.229,China,org,complete,yes,...,0.8,0,0.400000,0,0,2,0,0.000000,0.5,0
3,99,1,gmail.com>,http://hatchersmartialarts.homestead.com/front...,51,46.97.122.170,Romania,com,complete,yes,...,0.8,0,0.466667,0,1,2,2,0.071720,0.5,0
4,72,1,luebeck.de>,http://www.gabile.com/,22,94.145.85.24,Denmark,com,incomplete,no,...,0.4,0,0.357143,0,1,24,3,0.000000,0.5,0


In [12]:
df_encoded.head()

,email_subject_len,email_has_urgent_keyword,web_url_len,web_who_is,web_https,web_js_len,web_js_obf_len,domain_age,final_label,js_obfuscation_ratio,...,content_num_forms,content_num_inputs,content_num_scripts,content_suspicious_keywords,semantic_coherence_score,brand_consistency_score,domain_age_missing,email_domain_freq,geo_freq,tld_trust_score
0,32,0,34,1,1,137.0,0.00,11168.0,0,0.000000,...,0,0,5,0,0.043153,0.5,0,0.001278,0.424572,1.0
1,46,0,27,1,1,94.0,0.00,9692.0,0,0.000000,...,0,0,2,0,0.081125,0.5,0,0.071976,0.057658,1.0
2,21,0,25,1,1,44.5,0.00,2344.0,0,0.000000,...,0,0,2,0,0.000000,0.5,0,0.006009,0.094861,1.0
3,99,1,51,1,1,84.5,0.00,10335.0,0,0.000000,...,0,1,2,2,0.071720,0.5,0,0.071976,0.002046,1.0
4,72,1,22,0,0,837.0,460.35,7421.0,1,0.549344,...,0,1,24,3,0.000000,0.5,0,0.000128,0.004858,1.0


In [13]:
print(df_encoded.head())

   email_subject_len  email_has_urgent_keyword  web_url_len  web_who_is  \
0                 32                         0           34           1   
1                 46                         0           27           1   
2                 21                         0           25           1   
3                 99                         1           51           1   
4                 72                         1           22           0   

   web_https  web_js_len  web_js_obf_len  domain_age  final_label  \
0          1       137.0            0.00     11168.0            0   
1          1        94.0            0.00      9692.0            0   
2          1        44.5            0.00      2344.0            0   
3          1        84.5            0.00     10335.0            0   
4          0       837.0          460.35      7421.0            1   

   js_obfuscation_ratio  ...  content_num_forms  content_num_inputs  \
0              0.000000  ...                  0                

In [6]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('final_label', axis=1)
y = df_encoded['final_label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(X_train.shape, X_test.shape)


(6257, 36) (1565, 36)


In [7]:
# Count classes in training set
import pandas as pd

print("Training set class distribution:")
print(y_train.value_counts())

print("\nTest set class distribution:")
print(y_test.value_counts())


Training set class distribution:
final_label
0    3200
1    3057
Name: count, dtype: int64

Test set class distribution:
final_label
0    800
1    765
Name: count, dtype: int64


In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)


In [9]:
# First 5 rows of scaled training data
print("Scaled Training Data:")
display(X_train_scaled.head())

# First 5 rows of scaled test data
print("\nScaled Test Data:")
display(X_test_scaled.head())


Scaled Training Data:


,email_subject_len,email_has_urgent_keyword,web_url_len,web_who_is,web_https,web_js_len,web_js_obf_len,domain_age,js_obfuscation_ratio,url_has_ip,...,content_num_forms,content_num_inputs,content_num_scripts,content_suspicious_keywords,semantic_coherence_score,brand_consistency_score,domain_age_missing,email_domain_freq,geo_freq,tld_trust_score
0,0.136842,1.0,0.126354,1.0,1.0,0.152207,0.000000,0.547386,0.000000,0.0,...,0.0,0.142857,0.000000,0.25,0.322002,1.0,0.0,1.000000,1.000000,1.0
1,0.098246,1.0,0.003610,1.0,1.0,0.180307,0.000000,0.538528,0.000000,0.0,...,0.0,0.000000,0.083333,0.50,0.000000,1.0,0.0,1.000000,1.000000,0.0
2,0.087719,0.0,0.021661,0.0,0.0,0.539515,0.317010,0.035365,0.584526,0.0,...,0.0,0.142857,0.333333,0.25,0.782408,1.0,0.0,0.000000,0.069277,1.0
3,0.070175,1.0,0.212996,0.0,1.0,0.950474,0.934189,0.000000,0.978669,0.0,...,0.0,0.285714,0.541667,0.75,0.223506,1.0,1.0,0.000000,0.029819,1.0
4,0.189474,0.0,0.119134,1.0,1.0,0.206650,0.000000,0.014252,0.000000,0.0,...,0.0,0.000000,0.083333,0.75,0.000000,1.0,0.0,0.007117,1.000000,1.0



Scaled Test Data:


,email_subject_len,email_has_urgent_keyword,web_url_len,web_who_is,web_https,web_js_len,web_js_obf_len,domain_age,js_obfuscation_ratio,url_has_ip,...,content_num_forms,content_num_inputs,content_num_scripts,content_suspicious_keywords,semantic_coherence_score,brand_consistency_score,domain_age_missing,email_domain_freq,geo_freq,tld_trust_score
0,0.112281,1.0,0.043321,0.0,1.0,0.227725,0.000000,0.680386,0.000000,0.0,...,0.0,0.142857,0.166667,0.75,0.00000,1.0,0.0,0.007117,0.223193,1.0
1,0.014035,1.0,0.119134,0.0,0.0,0.730242,0.647518,0.013054,0.882602,0.0,...,0.0,0.428571,0.708333,0.75,0.29652,1.0,0.0,0.000000,1.000000,1.0
2,0.238596,1.0,0.057762,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.00,0.00000,1.0,1.0,0.026690,1.000000,1.0
3,0.122807,0.0,0.111913,0.0,0.0,0.839831,0.556276,0.000000,0.659431,0.0,...,0.0,0.571429,0.833333,0.75,0.47363,1.0,1.0,0.001779,1.000000,1.0
4,0.140351,0.0,0.090253,0.0,0.0,0.900948,0.837386,0.000000,0.925418,0.0,...,0.0,0.428571,0.708333,0.50,0.00000,1.0,1.0,0.001779,0.080723,1.0


In [10]:
#FEATURE SELECTION ON TRAIN DATA

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(
    X_train_scaled,
    y_train,
    random_state=42
)

mi_df = pd.DataFrame({
    'feature': X_train.columns,
    'mi_score': mi_scores
}).sort_values(by='mi_score', ascending=False)

mi_df.head(15)


,feature,mi_score
5,web_js_len,0.657304
8,js_obfuscation_ratio,0.521160
6,web_js_obf_len,0.519802
28,content_num_scripts,0.494982
22,content_entropy,0.472273
23,domain_trust_score,0.369185
33,email_domain_freq,0.324097
4,web_https,0.276868
29,content_suspicious_keywords,0.269552
3,web_who_is,0.261283


In [11]:
top_k = 6
top_features = mi_df.head(top_k)['feature'].tolist()

X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

print("Selected features:", top_features)
print(X_train_selected.shape, X_test_selected.shape)


Selected features: ['web_js_len', 'js_obfuscation_ratio', 'web_js_obf_len', 'content_num_scripts', 'content_entropy', 'domain_trust_score']
(6257, 6) (1565, 6)


In [12]:
#CORRECTED IMPORTS

# Qiskit imports (CORRECTED for 2025)
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.primitives import Sampler
from qiskit_aer import AerSimulator

print("📦 All libraries imported successfully!")
print("✅ Using FidelityQuantumKernel (current API)")

📦 All libraries imported successfully!
✅ Using FidelityQuantumKernel (current API)


In [13]:
# BATCH-WISE SVM & QSVM

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time
from qiskit.visualization import circuit_drawer
from sklearn.utils import shuffle

algorithm_globals.random_seed = 42

feature_map = ZZFeatureMap(
    feature_dimension=top_k,
    reps=1,
    entanglement="linear"
)
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

print(f"✅ Quantum kernel created")
print(f"   • Circuit depth: ~{feature_map.depth()}")

# Display & save the quantum circuit
if top_k <= 11:
    print(f"\n🔬 Quantum Feature Map Circuit:")
    print(feature_map.decompose().draw(output='text'))

else:
    print(f"\n🔬 Quantum circuit created (too large to display: {top_k} qubits)")


BATCH_SIZE = 500

print(f"Number of batches: {len(X_train_selected) // BATCH_SIZE + 1}")

# SHUFFLE DATA BEFORE BATCHING

print("\n🔀 Shuffling training data to ensure balanced batches...")

X_train_shuffled, y_train_shuffled = shuffle(
    X_train_selected,
    y_train,
    random_state=42
)

print("✅ Data shuffled")

# SCALE DATA AGAIN WITH PI(SAME FOR BOTH MODELS)
print("\n📊 Scaling data for both SVM and QSVM...")

# Scale training data to [0, pi] for quantum compatibility
X_train_scaled_q = X_train_shuffled.values * np.pi
X_test_scaled_q = X_test_selected.values * np.pi

print("✅ Both models will use pi-scaled features")

# BATCH-WISE TRAINING

print("STARTING BATCH-WISE TRAINING")

svm_preds_list = []
qsvm_preds_list = []

total_batches = 0
skipped_batches = 0

for i in range(0, len(X_train_shuffled), BATCH_SIZE):
    batch_num = i // BATCH_SIZE + 1
    total_batches += 1
    
    # Extract batch
    X_batch = X_train_scaled_q[i:i+BATCH_SIZE]
    y_batch = y_train_shuffled.iloc[i:i+BATCH_SIZE].values
    
    # Check class distribution in batch
    unique_classes, class_counts = np.unique(y_batch, return_counts=True)
    class_dist = dict(zip(unique_classes, class_counts))
    
    
    print(f"BATCH {batch_num}/{len(X_train_shuffled) // BATCH_SIZE + 1}")
    
    print(f"Samples: {len(X_batch)}")
    print(f"Class distribution: {class_dist}")
    
    # Skip batch if only one class present
    if len(unique_classes) < 2:
        print(f" SKIPPED: Only one class in batch")
        skipped_batches += 1
        continue
    
    # Calculate class balance ratio
    balance_ratio = min(class_counts) / max(class_counts)
    print(f"Balance ratio: {balance_ratio:.3f}")

    # Train Classical SVM
    print(f"\n🔵 Training Classical SVM on batch {batch_num}...")
    svm_start = time.time()
    
    svm = SVC(kernel='rbf', class_weight='balanced', random_state=42)
    svm.fit(X_batch, y_batch)
    
    svm_time = time.time() - svm_start
    svm_pred = svm.predict(X_test_scaled_q)
    svm_preds_list.append(svm_pred)
    
    print(f"✅ SVM trained in {svm_time:.2f}s")
    

    # Train QSVM
    
    print(f"⚛️  Training QSVM on batch {batch_num}...")
    qsvm_start = time.time()
    
    qsvm = QSVC(
        quantum_kernel=quantum_kernel,
        C=1.0
    )
    
    try:
        qsvm.fit(X_batch, y_batch)
        qsvm_time = time.time() - qsvm_start
        
        qsvm_pred = qsvm.predict(X_test_scaled_q)
        qsvm_preds_list.append(qsvm_pred)
        
        print(f"✅ QSVM trained in {qsvm_time:.2f}s ({qsvm_time/60:.2f} min)")
        print(f"⏱️  Speed ratio: QSVM is {qsvm_time/svm_time:.1f}x slower than SVM")
        
    except Exception as e:
        print(f"❌ QSVM training failed: {e}")
        skipped_batches += 1
        # Remove corresponding SVM prediction
        svm_preds_list.pop()
        continue
    

✅ Quantum kernel created
   • Circuit depth: ~1

🔬 Quantum Feature Map Circuit:
     ┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■───────»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐     »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──■──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     ├───┤├─────────────┤                                          └───┘»
q_3: ┤ H ├┤ P(2.0*x[3]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_4: ┤ H ├┤ P(2.0*x[4]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_5: ┤ H ├┤ P(2.0*x[5]) ├───────────────────────────────────────────────»
     └───┘└─────────────┘       

In [14]:
# ENSEMBLE PREDICTIONS

print("GENERATING ENSEMBLE PREDICTIONS")


print(f"\nTotal batches processed: {total_batches}")
print(f"Skipped batches: {skipped_batches}")
print(f"Successful batches: {len(svm_preds_list)}")

# Majority voting ensemble
svm_ensemble = np.array(svm_preds_list).T
qsvm_ensemble = np.array(qsvm_preds_list).T

from scipy import stats

svm_final_pred = stats.mode(svm_ensemble, axis=1, keepdims=False)[0]
qsvm_final_pred = stats.mode(qsvm_ensemble, axis=1, keepdims=False)[0]

print("✅ Ensemble predictions computed using majority voting")


# EVALUATION

print("FINAL RESULTS")
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("\n🔵 CLASSICAL SVM (Batch Ensemble):")

svm_acc = accuracy_score(y_test, svm_final_pred)
print(f"Accuracy: {svm_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, svm_final_pred))
print("\nClassification Report:")
print(classification_report(y_test, svm_final_pred))

print("\n" + "=" * 60)
print("⚛️  QUANTUM SVM (Batch Ensemble):")
print("─" * 40)
qsvm_acc = accuracy_score(y_test, qsvm_final_pred)
print(f"Accuracy: {qsvm_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, qsvm_final_pred))
print("\nClassification Report:")
print(classification_report(y_test, qsvm_final_pred))



GENERATING ENSEMBLE PREDICTIONS

Total batches processed: 13
Skipped batches: 0
Successful batches: 13
✅ Ensemble predictions computed using majority voting
FINAL RESULTS

🔵 CLASSICAL SVM (Batch Ensemble):
Accuracy: 0.9770

Confusion Matrix:
[[793   7]
 [ 29 736]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       800
           1       0.99      0.96      0.98       765

    accuracy                           0.98      1565
   macro avg       0.98      0.98      0.98      1565
weighted avg       0.98      0.98      0.98      1565


⚛️  QUANTUM SVM (Batch Ensemble):
────────────────────────────────────────
Accuracy: 0.9879

Confusion Matrix:
[[787  13]
 [  6 759]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       800
           1       0.98      0.99      0.99       765

    accuracy                           0.99      1565
   macr

In [15]:
# COMPARISON
print("📊 MODEL COMPARISON")

print(f"Classical SVM Accuracy: {svm_acc:.4f}")
print(f"Quantum SVM Accuracy:   {qsvm_acc:.4f}")
print(f"Difference:            {abs(qsvm_acc - svm_acc):.4f}")

if qsvm_acc > svm_acc:
    print("✅ QSVM outperforms classical SVM")
elif qsvm_acc < svm_acc:
    print("⚠️  Classical SVM outperforms QSVM")
else:
    print("🤝 Both models perform equally")


📊 MODEL COMPARISON
Classical SVM Accuracy: 0.9770
Quantum SVM Accuracy:   0.9879
Difference:            0.0109
✅ QSVM outperforms classical SVM
